# Upper Slope & Axis 75 kHz ADCP

To create various plots for either the Upper Slope or Axis ADCP velocity data.

## Imports

In [2]:
import xarray as xr
import datetime
from datetime import datetime
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.dates as pldates
import numpy as np
import scipy.signal as sig
import scipy.interpolate as interp
import pandas as pd
from scipy.stats import chi2
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib notebook

## Data

In [95]:
#ds = xr.open_dataset('../Data/SlopeAll/slope_all.nc')
ds = xr.open_dataset('../Data/AxisAll/axis_all.nc')
print(ds)

<xarray.Dataset>
Dimensions:    (depth: 422, latitude: 4, longitude: 4, time: 162119)
Coordinates:
  * depth      (depth) float32 336.2703 336.27032 ... 968.27594 970.27026
  * latitude   (latitude) float32 48.316517 48.3166 48.316742 48.316772
  * longitude  (longitude) float32 -126.0508 -126.05022 -126.05016 -126.050125
  * time       (time) datetime64[ns] 2012-06-02T05:07:29.999996672 ... 2018-06-22T10:52:30
Data variables:
    u          (time, depth) float32 ...
    v          (time, depth) float32 ...
    w          (time, depth) float32 ...
    temp       (time) float32 ...
Attributes:
    Conventions:                             CF-1.6
    title:                                   Ocean Networks Canada RDI ADCP Data
    institution:                             Ocean Networks Canada
    source:                                  Fixed-position Teledyne-RDI ADCP...
    history:                                 data extracted from raw output, ...
    references:                       

## Depth

Truncate data to a specific depth interval to eliminate unreliable data (from visual inspection of velocity plots) from the top and bottom of the ADCP beam.

In [96]:
# function to find nearby indices for desired depth values
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx     # returns index of nearest depth value

array = ds.depth   # input array to process 
#upval = 100        # upper depth for data, metres (upper slope)
#lowval = 330       # lower depth for data (upper slope)
upval = 400        # upper depth for data, metres (axis)
lowval = 900       # lower depth for data (axis)
upidx = find_nearest(array, upval)               # index of upper depth cutoff
lowidx = find_nearest(array,lowval)+1              # index of lower depth cutoff 

print("Index at upper depth cutoff:", upidx)
print("Value at upper depth cutoff:", -ds.depth.values[upidx], "metres" )
print('---')
print("Index at lower depth cutoff:", lowidx)
print("Value at lower depth cutoff:", -ds.depth.values[lowidx], "metres" )

depth = np.array(ds.depth[upidx:lowidx+1])       # remove unwanted depths, this will be used in the next steps

print('---')
print("Length of new depth array: ", len(depth)) # new depth interval
print('Upper limit at',-depth[0],'metres')      # depth for upper PSD
print('Lower limit at',-depth[-1],'metres')       # depth for lower PSD

# depth stamps for use in output filenames
dup_stamp = int(-depth[0])
dlow_stamp = int(-depth[-1])

Index at upper depth cutoff: 48
Value at upper depth cutoff: -400.2703 metres
---
Index at lower depth cutoff: 377
Value at lower depth cutoff: -904.2703 metres
---
Length of new depth array:  330
Upper limit at -400.2703 metres
Lower limit at -904.2703 metres


## Time

Find specific time range and format dates. If significant *consecutive* NaN values are present then shorten series for that particular spectrum.

In [133]:
datestimes = pd.to_datetime(ds.time.values)        # convert to datetime from datetime64
datestimes = pd.Series(datestimes)                 # convert to pandas dataframe

start_date = pd.datetime(2018,1,1)
end_date = pd.datetime(2018,6,22)
start = datestimes[datestimes >= start_date].index[0]     # desired start date
end = datestimes[datestimes < end_date].index[-1]       # desired end date
time_total = ds.time.values[start:end]                               # total interval
print("Initial time range:",np.min(time_total),np.max(time_total))   # print to check desired interval

# check new time series for significant NaN values
depth_test = ds.depth[lowidx]     # mid-depth for ideal data (less noise, etc.)   
print('at depth',ds.depth.values[lowidx],'m')
u_test = np.array(ds.u[start:end,lowidx])        # u data at this depth
NaN_series = np.zeros(len(u_test))                   # empty array to indicate NaN values
counter = 0                                          # counter to keep track of # of consecutive NaN values
for i in range(len(time_total)):                     # loop to count consecutive NaN values
    if np.isnan(u_test[i])==True:                    # add to counter if NaN = true
        NaN_series[i] = 1
        counter += 1
        if counter==2688:                              # a consecutive month worth of NaN
            dead = i-2688                              # date time series hits significant NaN interval
            print("Time series hits trouble AFTER date:",time_total[dead])
            time_new = time_total[0:dead]            # new truncated interval
            end -= (len(time_total)-len(time_new))   # new end date 
            print("Good data time range:",np.min(time_new),np.max(time_new))
    elif np.isnan(u_test[i])==False:                 # reset counter if NaN inconsistent
        counter = 0       

# usable interval, for next steps
time = ds.time.values[start:end]    

# set year time stamp for output filenames
t_stamp = f'{datestimes.dt.year[start]}'

Initial time range: 2018-01-01T00:07:30.000003328 2018-06-21T23:37:30.000000000
at depth 904.2703 m
Time series hits trouble AFTER date: 2018-06-21T23:37:30.000000000
Good data time range: 2018-01-01T00:07:30.000003328 2018-06-21T23:22:29.999993344


/Users/Kurtis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.
/Users/Kurtis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


## Filters

Digital low-pass Butterworth filter to remove tides, if necessary.

In [134]:
# low pass Butterworth filter for 40 hour cut-off to remove 30 hour tides

fs = 4                # samples per HOUR for entire time series
fc = 0.025            # 40 hour low pass filter cut-off
Wn = fc / (fs / 2)    # normalised cut-off frequencies
b, a = sig.butter(8, Wn,'lowpass')  # digital butterworth filter
w, h = sig.freqz(b, a)

## Rotate, interpolate

Loop to acquire rotated and NaN filtered data.

Data rotated based on a visual estimate of along-slope angle, as 30$^{\circ}$. This could be updated to reflect Thomson's work at the A1 site, relatively nearby.

Data are also interpolated to deal with minor instances of NaN values. Consistent NaN intervals are dealt with in the Time section, above.

In [135]:
# rotate data
theta_along_slope = np.radians(30)                       # rotation angle in radians, 30 degrees
u_vec = ds.u[start:end,upidx:lowidx+1] + 1j*ds.v[start:end,upidx:lowidx+1]  # vector form of horizontal velocity
u_vec_new = u_vec*np.exp(-1j*theta_along_slope)          # rotated velocity vector
u_rot = np.real(u_vec_new)                               # u_new = Re(rotated vector)
v_rot = np.imag(u_vec_new)                               # v_new = Im(rotated vector)

# filter NaN instances from data
t = len(time)                  # number of time data points after checking for consistent NaN intervals
d = len(depth)                 # number of depth data points after removing unwanted depths

uorig = np.empty([t,d])        # empty array for rotated u data
vorig = np.empty([t,d])        # empty array for rotated v data

for j in range(d):                       # loop to filter NaN instances from velocities at each depth
    utemp = pd.Series(u_rot[:,j])
    uint = utemp.interpolate(method="cubic")
    uorig[:,j] = uint                    # set interpolated data to original array
    
    vtemp = pd.Series(v_rot[:,j])
    vint = vtemp.interpolate(method="cubic")
    vorig[:,j] = vint                    # set interpolated data to original array
    
for i in range(t):                       # loop to filter NaN instances from depth values
    utemp = pd.Series(uorig[i,:])
    uint = utemp.interpolate(method="linear", limit_direction="both")
    uorig[i,:] = uint
    
    vtemp = pd.Series(vorig[i,:])
    vint = vtemp.interpolate(method="linear", limit_direction="both")
    vorig[i,:] = vint
    
ulp = np.empty([t,d])          # empty array for low-pass filtered u values
vlp = np.empty([t,d])          # empty array for low-pass filtered v values
uhp = np.empty([t,d])          # empty array for residual u values
vhp = np.empty([t,d])          # empty array for residual v values

for j in range(d):
    uint = uorig[:,j]
    ulp[:,j] = sig.filtfilt(b, a, uint)
    uhp[:,j] = uint - ulp[:,j]
    
    vint = vorig[:,j]
    vlp[:,j] = sig.filtfilt(b, a, vint)
    vhp[:,j] = vint - vlp[:,j]

# Spectra

## Welch FFT

Remove the mean from upper and lower depth limits, for spectra at those depths.

Also obtain depth mean data for an average spectra through depth.

Perform Welch FFT with adjustable parameters, using a Parzen window and what seems to be an optimal averaging process, based on visual analysis of spectral output.

Calculate 95% confidence intervals using a chi$^2$ based method.

## Spectrogram

Creates a spectrogram for the rotated, cleaned, and mean-removed velocity data. Spectrogram has been 'whitened', and parameters adjusted for optimal visual clarity.

## Rotary

Creates rotary spectra for the mean-removed, cleaned, and NON-rotated velocity data.

This isn't working. Follow Rick's book, ch.5.

## PSD

# Velocities